In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.colors as colors
import pickle

In [ ]:
hMinus = 48
hPlus = 47
hMinusIQR = 96
hPlusIQR = 95
hMinusHourly = 24*20 # 24 hours by 20 days
hPlusHourly = 24*20-1 # 24 hours by 20 days - 1 hour gives 40 days total
nDays = 20
# based on Pandas slicing

def add_rolling_dem(df):
    # Can't use np.roll b/c it does not deal with NANs
    # in a sophisticated manner.  Use np.nanmean which
    # skips all NANs and leaves them out of the sum and
    # division
    rolling = np.empty((0,), float)
    
    for i in range(len(df.index)):
        val = np.nanmedian(df.loc[i-hMinus:i+hPlus, 'demand (MW)'])
        rolling = np.append(rolling, val)
    
    return df.assign(rollingDem=rolling)



def add_demand_minus_rolling_dem(df):
    diff = df['demand (MW)'] - df['rollingDem']
    df = df.assign(dem_minus_rolling=diff)
    return df



def add_demand_minus_rolling_dem_iqr(df):
    rolling = np.empty((0,), float)
    
    for i in range(len(df.index)):
        lst = df.loc[i-hMinusIQR:i+hPlusIQR, 'dem_minus_rolling']
        iqr = np.nanpercentile(lst, 75) - np.nanpercentile(lst, 25)
        rolling = np.append(rolling, iqr)
    
    return df.assign(dem_minus_rolling_IQR=rolling)



def add_hourly_median_dem_deviations(df):
    """
    hourly = np.empty((0,), float)

    for i in range(len(df.index)):
        low = max(0, i-hMinusHourly)
        high = min(len(df.index)-1,i+hPlusHourly)
        df_slice = df.loc[low:high]
        hourly_vals = []
        i_hour = df_slice.at[i, 'time'].hour
        for j in range(low, high+1):
            if df_slice.at[j, 'time'].hour == i_hour:
                hourly_vals.append(df_slice.at[j, 'dem_minus_rolling'])
        hourly = np.append(hourly, np.nanmedian(hourly_vals)/np.nanmedian(df_slice['rollingDem']))
        if i%500==0:
            print("add_hourly_median_dem_deviations - {}".format(i))

    return df.assign(hourly_median_dem_dev=hourly)
    """
    cnts_dem_minus_rolling = df['dem_minus_rolling'].notna().astype(int)
    cnts_rollingDem = df['rollingDem'].notna().astype(int)
    vals_dem_minus_rolling = df['dem_minus_rolling'].dropna()
    vals_rollingDem = df['rollingDem'].dropna()
    # Loop over 20 days on each side
    for i in range(-nDays, nDays+1):
        # Already initialized with zero value
        if i == 0:
            continue
        cnts_dem_minus_rolling += df.shift(periods=i*24)['dem_minus_rolling'].notna().astype(int)
        cnts_rollingDem += df.shift(periods=i*24)['rollingDem'].notna().astype(int)
        vals_dem_minus_rolling += df.shift(periods=i*24)['dem_minus_rolling'].dropna()
        vals_rollingDem += df.shift(periods=i*24)['rollingDem'].dropna()
    #print("cnts_dem_minus_rolling",cnts_dem_minus_rolling.loc[5000:5010])
    #print("cnts_rollingDem",cnts_rollingDem.loc[5000:5010])
    #print("vals_dem_minus_rolling",vals_dem_minus_rolling.loc[5000:5010])
    #print("vals_rollingDem",vals_rollingDem.loc[5000:5010])
    vals_dem_minus_rolling /= cnts_dem_minus_rolling
    vals_rollingDem /= cnts_rollingDem
    vals_dem_minus_rolling /= vals_rollingDem
    return df.assign(hourly_median_dem_dev=vals_dem_minus_rolling)

                
                
# delta with previous and following time steps
def add_deltas(df):
    diff = df['demand (MW)'].diff()
    df = df.assign(diff_pre=diff)
    diff = df['demand (MW)'].diff(periods=-1)
    df = df.assign(diff_post=diff)
    return df

def add_rolling_diff_iqr(df):
    rolling = np.empty((0,), float)
    
    for i in range(len(df.index)):
        lst = df.loc[i-hMinusIQR:i+hPlusIQR, 'diff_pre']
        iqr = np.nanpercentile(lst, 75) - np.nanpercentile(lst, 25)
        rolling = np.append(rolling, iqr)
    
    return df.assign(rollingDiffIQR=rolling)


    
def filter_extrem_demand(df, multiplier):
    med = np.nanmedian(df['demand (MW)'])
    filtered = df['demand (MW)'].where(df['demand (MW)'] < med * multiplier)
    df['globalDemandFiltered'] = np.where(df['demand (MW)'] != filtered, df['demand (MW)'], np.nan)
    df['demand (MW)'] = filtered
    return df
    

def filter_local_demand(df, iq2, iq3, multiplier):
    dem_iqr = iq3 - iq2
    
    filtered = df['demand (MW)'].where((df['dem_diff_norm_rolling'] < dem_iqr * multiplier) & \
                                      (df['dem_diff_norm_rolling'] > -1. * dem_iqr * multiplier))
    df['demandFiltered'] = np.where(df['demand (MW)'] != filtered, df['demand (MW)'], np.nan)
    df['demand (MW)'] = filtered
    return df


# Filter on a multiplier of the IQR and set
# the associated 'demand (MW)' value to NAN.
# Filter on 1 multiplier for double deltas
# and another for single jumps
def filter_deltas(df, iq2, iq3, iqr, m_double):#, m_single):
    
    filtered = df['demand (MW)'].where(
            (((df['diff_pre_norm_diffIQR'] < iqr * m_double) & \
            (df['diff_pre_norm_diffIQR'] > -1. * iqr * m_double)) | \
             df['diff_pre_norm_diffIQR'].isna()) & \
            (((df['diff_post_norm_diffIQR'] < iqr * m_double) & \
            (df['diff_post_norm_diffIQR'] > -1. * iqr * m_double)) | \
             df['diff_post_norm_diffIQR'].isna()))
            
    df['deltaFiltered'] = np.where(df['demand (MW)'] != filtered, df['demand (MW)'], np.nan)
    df['demand (MW)'] = filtered
    return df


def filter_runs(df):
    
    d1 = df['demand (MW)'].diff(periods=1)
    d2 = df['demand (MW)'].diff(periods=2)
    print(d1)
    # cannot compare a dtyped [float64] array with a scalar of type [bool]
    filtered = df['demand (MW)'].mask((d1 == 0) & (d2 == 0))
    df['runFiltered'] = np.where(df['demand (MW)'] != filtered, df['demand (MW)'], np.nan)
    df['demand (MW)'] = filtered
    return df
    
    

def mark_missing_and_empty(df, col):
    #marked = np.zeros(len(df.index))
    print(df[col].isna())

def show_structure(df):
    plt.imshow(~df.isna(), aspect='auto')
    plt.xlabel("variables")
    plt.ylabel("cases")
    plt.gray()
    plt.show()


def simple_hist(col, df, iq2, iq3, factor, save, x_log=False):
    plt.close()
    fig, ax = plt.subplots()
    
    if df[col].max() == np.Inf:
        print(save, df[col].max())
        return
    if df[col].min() == np.NINF:
        print(save, df[col].min())
        return
    n, bins, patches = ax.hist(df[col] * (~df['demand (MW)'].isna()), 100, facecolor='red', alpha=0.2, label='pre')
    #n, bins, patches = ax.hist(df['diff_post'], 100, facecolor='blue', alpha=0.2, label='post')
    if col == 'Demand (MW)':
        ax.set_xlabel('Demand (MW)')
    elif col == 'dem_diff_norm_rolling':
        ax.set_xlabel('$\Delta$(Demand, Rolling Avg)/Rolling IQR')
    elif col == 'dem_minus_rolling':
        ax.set_xlabel('$\Delta$(Demand, Rolling Avg) (MW)')
    elif col == 'diff_pre':
        ax.set_xlabel('$\Delta$(Demand ti, Demand ti-1) (MW)')
    elif col == 'diff_pre_norm':
        ax.set_xlabel('Normalized Demand Difference (diff/Rolling IQR)')
    elif col == 'diff_norm_diffIQR_D':
        ax.set_xlabel('Normalized Demand Difference ($\Delta$(t-1, t+1)/Rolling IQR)')
    ax.set_ylabel('Counts')
            
    # Draw iq2 and iq3
    iqr = iq3 - iq2
    iq2_l1 = mlines.Line2D([-iqr,-iqr], ax.get_ylim())
    ax.add_line(iq2_l1)
    iq2_l2 = mlines.Line2D([-iqr*factor,-iqr*factor], ax.get_ylim())
    ax.add_line(iq2_l2)
    iq3_l1 = mlines.Line2D([iqr,iqr], ax.get_ylim())
    ax.add_line(iq3_l1)
    iq3_l2 = mlines.Line2D([iqr*factor,iqr*factor], ax.get_ylim())
    ax.add_line(iq3_l2)
    
    if x_log:
        plt.xscale('log', nonposx='clip')
    plt.tight_layout()
    plt.yscale('log', nonposy='clip')
    plt.savefig(save)
    
    



# Create many demand plots so we can actually see the values
def scrolling_demand(width, df, title, save):
    start = 0
    end = width
    i = 0
    tot_l = len(df.index)
    while True:
        s = save.replace('.png', '_{}cnt'.format(i))
        t = title+': cnt {}'.format(i)
        o = df.loc[start:end]

        # end-start+1 is the length, remember pandas slice notation includes end point
        if not (df['demandFiltered'].loc[start:end].isna().sum() == len(o.index) and \
                df['deltaFiltered'].loc[start:end].isna().sum() == len(o.index)):
            comparison_demand_plot(o, t, s)
            comparison_diff_plot(o, t, s.replace('cnt', 'cnt_diff'))
        if end == tot_l:
            break
        i += 1
        start += width
        end += width
        if end >= tot_l:
            end = tot_l

def comparison_demand_plot(df, title, save):
    plt.close()
    fig, ax = plt.subplots(figsize=(15,5))
    ax.set_xlabel('Hour')
    ax.set_ylabel('Demand')
    plt.title(title)
    ax.plot(df['demand (MW)'], 'r-', label='demand')
    ax.plot(df['rollingDem'], 'b-', label='rolling dem+/-')
    ax.plot(df['rollingDem']+df['dem_minus_rolling_IQR'], 'b-')
    ax.plot(df['rollingDem']-df['dem_minus_rolling_IQR'], 'b-')
    ax.plot(df['demandFiltered'], 'mo', label='demandFiltered')
    ax.plot(df['deltaFiltered'], 'go', label='deltaFiltered')
    plt.legend()
    plt.tight_layout()
    plt.savefig(save)
    
def comparison_diff_plot(df, title, save):
    plt.close()
    fig, ax = plt.subplots(figsize=(15,5))
    ax.set_xlabel('Hour')
    ax.set_ylabel('$\Delta$(Demand ti, ti-1) (MW)')
    plt.title(title)
    ax.plot(df['diff_pre_norm_diffIQR'], 'b-', label='diff_pre_norm_diffIQR')
    ax.plot(df['rollingDiffNormIQR_IQR'], 'b--')
    ax.plot(-df['rollingDiffNormIQR_IQR'], 'b--')
    ax.plot(3*df['rollingDiffNormIQR_IQR'], 'b-.')
    ax.plot(-3*df['rollingDiffNormIQR_IQR'], 'b-.')
    plt.legend()
    plt.tight_layout()
    plt.savefig(save)

    
    
def get_iqrs(vals):
    iq3 = np.nanpercentile(vals, 75)
    iq2 = np.nanpercentile(vals, 25)
    iqr = iq3 - iq2
    return iqr, iq2, iq3


def return_all_regions():
    return ['AEC', 'AECI', 'CPLE', 'CPLW',
    'DUK', 'FMPP', 'FPC',
    'FPL', 'GVL', 'HST', 'ISNE',
    'JEA', 'LGEE', 'MISO', 'NSB',
    'NYIS', 'OVEC', 'PJM', 'SC',
    'SCEG', 'SEC', 'SOCO',
    'SPA', 'SWPP', 'TAL', 'TEC',
    'TVA', 'ERCO',
    'AVA', 'AZPS', 'BANC', 'BPAT',
    'CHPD', 'CISO', 'DOPD',
    'EPE', 'GCPD', 'IID',
    'IPCO', 'LDWP', 'NEVP', 'NWMT',
    'PACE', 'PACW', 'PGE', 'PNM',
    'PSCO', 'PSEI', 'SCL', 'SRP',
    'TEPC', 'TIDC', 'TPWR', 'WACM',
    'WALC', 'WAUW']

In [ ]:
dem_map = {}
regions = ['TIDC', 'CISO', 'LDWP', 'BANC']
regions = ['LDWP',]# 'CISO', 'LDWP']
regions = ['CISO',]
dump_to_pickle = True
load_from_pickle = False
do_timing = False
resultsM = {
    'region' : [],
    'diff_mean' : [],
    'diff_iqr' : [],
    'diff_iq2' : [],
    'diff_iq3' : [],
    'diff_within' : [],
    'diff_outside' : [],
    'dem_mean' : [],
    'dem_iqr' : [],
    'dem_iq2' : [],
    'dem_iq3' : [],
    'dem_within' : [],
    'dem_outside' : [],
}
results = {}
results_only = True
if results_only:
    print("\nOnly returning results, not many plots\n")
o_file = open('results_dump.csv', 'w')
for region in regions:
#for region in return_all_regions():
    print(region)
    results_v = []
    dem_map = {}
    if not load_from_pickle:
        file_path = '../get_eia_demand_data/data/{}.csv'.format(region)
        dem_map[region] = pd.read_csv(file_path, #index_col='time',
                           dtype={'demand (MW)':np.float64},
                          parse_dates=True, na_values=['MISSING', 'EMPTY'])

        # Convert date/time
        dem_map[region]['time'] = pd.to_datetime(dem_map[region]['time'])

        # Missing and empty values are marked
        dem_map[region] = dem_map[region].assign(missing=dem_map[region]['demand (MW)'].isna())

        # Set all negative and zero values to NAN
        dem_map[region]['demand (MW)'] = dem_map[region]['demand (MW)'].mask(dem_map[region]['demand (MW)'] <= 0.)

        # Set last demand values in runs of 3+ to NAN
        dem_map[region] = filter_runs(dem_map[region])
        print(dem_map[region].head(30))

        # Global demand filter on 10x the median value
        global_dem_cut = 10
        dem_map[region] = filter_extrem_demand(dem_map[region], global_dem_cut)
        
        if not do_timing:
            # Add rolling dem average
            dem_map[region] = add_rolling_dem(dem_map[region])
            dem_map[region] = add_demand_minus_rolling_dem(dem_map[region])
            dem_map[region] = add_hourly_median_dem_deviations(dem_map[region])
            dem_map[region] = add_demand_minus_rolling_dem_iqr(dem_map[region])
        
        
        
            # Add deltas
            dem_map[region] = add_deltas(dem_map[region])
            dem_map[region] = add_rolling_diff_iqr(dem_map[region])

        if do_timing:
            # Add rolling dem average
            print("add_rolling_dem")
            %time dem_map[region] = add_rolling_dem(dem_map[region])
            print("add_demand_minus_rolling_dem")
            %time dem_map[region] = add_demand_minus_rolling_dem(dem_map[region])
            print("add_hourly_median_dem_deviations")
            %time dem_map[region] = add_hourly_median_dem_deviations(dem_map[region])
            print("add_demand_minus_rolling_dem_iqr")
            %time dem_map[region] = add_demand_minus_rolling_dem_iqr(dem_map[region])
        
        
        
            # Add deltas
            print("add_deltas")
            %time dem_map[region] = add_deltas(dem_map[region])
            print("add_rolling_diff_iqr")
            %time dem_map[region] = add_rolling_diff_iqr(dem_map[region])

        
    
        if dump_to_pickle:
            print('Saving pickle /Users/truggles/tmp_data/pickle_{}.pkl'.format(region))
            pickle_file = open('/Users/truggles/tmp_data/pickle_{}.pkl'.format(region), 'wb') 
            pickle.dump(dem_map[region], pickle_file)
            pickle_file.close()
            continue
    
    if load_from_pickle:
        print('Loading from pickle /Users/truggles/tmp_data/pickle_{}.pkl'.format(region))
        pickle_in = open('/Users/truggles/tmp_data/pickle_{}.pkl'.format(region),'rb')
        dem_map[region] = pickle.load(pickle_in)
    
    # Calculate IQRs
    iqr, iq2, iq3 = get_iqrs(dem_map[region]['diff_pre'])
    n_iqr, n_iq2, n_iq3 = get_iqrs(dem_map[region]['diff_pre_norm'])
    n_iqr_diff, n_iq2_diff, n_iq3_diff = get_iqrs(dem_map[region]['diff_pre_norm_diffIQR'])
    dem_iqr, dem_iq2, dem_iq3 = get_iqrs(dem_map[region]['demand (MW)'])
    demD_iqr, demD_iq2, demD_iq3 = get_iqrs(dem_map[region]['dem_minus_rolling'])
    demDN_iqr, demDN_iq2, demDN_iq3 = get_iqrs(dem_map[region]['dem_diff_norm_rolling'])
    x_iqr, x_iq2, x_iq3 = get_iqrs(dem_map[region]['diff_norm_diffIQR_D'])
    
    multi = 2
    to_results = [
        region,
        np.nanmean(dem_map[region]['diff_pre_norm_diffIQR']), 
        n_iqr_diff, n_iq2_diff, n_iq3_diff,
        ((dem_map[region]['diff_pre_norm_diffIQR'].dropna() < -n_iqr_diff*multi) |
         (dem_map[region]['diff_pre_norm_diffIQR'].dropna() > n_iqr_diff*multi)).sum(),
        ((dem_map[region]['diff_pre_norm_diffIQR'].dropna() > -n_iqr_diff*multi) &
         (dem_map[region]['diff_pre_norm_diffIQR'].dropna() < n_iqr_diff*multi)).sum(),
        np.nanmean(dem_map[region]['dem_diff_norm_rolling']), 
        demDN_iqr, demDN_iq2, demDN_iq3,
        ((dem_map[region]['dem_diff_norm_rolling'].dropna() < -demDN_iqr*multi) |
         (dem_map[region]['dem_diff_norm_rolling'].dropna() > demDN_iqr*multi)).sum(),
        ((dem_map[region]['dem_diff_norm_rolling'].dropna() > -demDN_iqr*multi) &
         (dem_map[region]['dem_diff_norm_rolling'].dropna() < demDN_iqr*multi)).sum()
        ]
    resultsM['region'].append(to_results[0])
    resultsM['diff_mean'].append(to_results[1])
    resultsM['diff_iqr'].append(to_results[2])
    resultsM['diff_iq2'].append(to_results[3])
    resultsM['diff_iq3'].append(to_results[4])
    resultsM['diff_within'].append(to_results[5])
    resultsM['diff_outside'].append(to_results[6])
    resultsM['dem_mean'].append(to_results[7])
    resultsM['dem_iqr'].append(to_results[8])
    resultsM['dem_iq2'].append(to_results[9])
    resultsM['dem_iq3'].append(to_results[10])
    resultsM['dem_within'].append(to_results[11])
    resultsM['dem_outside'].append(to_results[12])
    

    for val in to_results:
        results_v.append(val)
    results[region] = results_v
    o_str = ''
    for val in results_v:
        o_str += str(val)+','
    o_file.write(o_str+'\n')
    if results_only:
        continue
    
    # Plots
    dem_multiplier = 5
    multiplier_double = 3
    #multiplier_single = 6
    dem_M = 4.
    #plot_2D_diff_and_demand_norm(dem_map[region], multiplier_double, dem_M, 'plt/{}_2D_diff_and_demand_norm_original.png'.format(region))
    
    logX = True if region == 'BANC' else False
    simple_hist('diff_pre_norm', dem_map[region], n_iq2, n_iq3, multiplier_double, 'plt/{}_diff_pre_norm_original.png'.format(region))
    simple_hist('diff_pre_norm_diffIQR', dem_map[region], n_iq2_diff, n_iq3_diff, multiplier_double, 'plt/{}_diff_pre_norm_diffIQR_original.png'.format(region))
    simple_hist('diff_pre', dem_map[region], iq2, iq3, multiplier_double, 'plt/{}_diff_pre_original.png'.format(region))
    simple_hist('demand (MW)', dem_map[region], dem_iq2, dem_iq3, dem_multiplier, 'plt/{}_demand_original.png'.format(region), logX)
    simple_hist('dem_minus_rolling', dem_map[region], demD_iq2, demD_iq3, dem_M, 'plt/{}_dem_minus_rolling_original.png'.format(region))
    simple_hist('dem_diff_norm_rolling', dem_map[region], demDN_iq2, demDN_iq3, dem_M, 'plt/{}_dem_diff_norm_rolling_original.png'.format(region))
    simple_hist('diff_norm_diffIQR_D', dem_map[region], x_iq2, x_iq3, dem_M, 'plt/{}_diff_norm_diffIQR_D_original.png'.format(region))
    
    

    dem_map[region] = filter_deltas(dem_map[region], n_iq2_diff, n_iq3_diff, n_iqr_diff, multiplier_double)#, multiplier_single)
    # Plot results for demand hist
    simple_hist('demand (MW)', dem_map[region], dem_iq2, dem_iq3, dem_multiplier, 
                  'plt/{}_dem_post-delta-filter.png'.format(region), logX)
    simple_hist('dem_minus_rolling', dem_map[region], demD_iq2, demD_iq3, dem_M, 'plt/{}_dem_minus_rolling_post_delta.png'.format(region))
    simple_hist('dem_diff_norm_rolling', dem_map[region], demDN_iq2, demDN_iq3, dem_M, 'plt/{}_dem_diff_norm_rolling_post_delta.png'.format(region))
    
    
    # Filter on extreme demand values
    dem_map[region] = filter_local_demand(dem_map[region], demDN_iq2, demDN_iq3, dem_M)
    
    simple_hist('demand (MW)', dem_map[region], dem_iq2, dem_iq3, dem_multiplier, 
                  'plt/{}_dem_post_ext_dem.png'.format(region))
    simple_hist('diff_pre_norm', dem_map[region], n_iq2, n_iq3, multiplier_double, 'plt/{}_diff_pre_norm_post_ext_dem.png'.format(region))
    simple_hist('diff_pre', dem_map[region], iq2, iq3, multiplier_double, 'plt/{}_diff_pre_post_ext_dem.png'.format(region))
    simple_hist('dem_minus_rolling', dem_map[region], demD_iq2, demD_iq3, dem_M, 'plt/{}_dem_minus_rolling_post_ext_dem.png'.format(region))
    simple_hist('dem_diff_norm_rolling', dem_map[region], demDN_iq2, demDN_iq3, dem_M, 'plt/{}_dem_diff_norm_rolling_post_ext_dem.png'.format(region))
    
    #plot_2D_diff_and_demand_norm(dem_map[region], multiplier_double, dem_M, 'plt/{}_2D_diff_and_demand_norm_final.png'.format(region))
    
    #print(dem_map[region].head())
    #print(dem_map[region].loc[1000:1005])

    plt.close()
    fig, ax = plt.subplots(figsize=(15,5))
    ax.plot(dem_map[region]['demand (MW)'], 'r-', label='demand')
    ax.plot(dem_map[region]['rollingDem'], 'b-', label='rolling dem')
    plt.tight_layout()
    plt.legend()
    plt.title("{} Cleaned Demand".format(region))
    plt.savefig('plt/{}_demand.png'.format(region))
    
    plt.close()
    fig, ax = plt.subplots(figsize=(15,5))
    ax.plot(dem_map[region]['demand (MW)'], 'r-', label='demand')
    ax.plot(dem_map[region]['rollingDem'], 'b-', label='rolling dem')
    ax.plot(dem_map[region]['demandFiltered'], 'g-', label='demandFiltered')
    ax.plot(dem_map[region]['deltaFiltered'], 'y-', label='demandFiltered')
    plt.tight_layout()
    plt.legend()
    plt.title("{} Cleaned Demand".format(region))
    plt.savefig('plt/{}_demand_show_filters.png'.format(region))
    
    width = 500
    title = '{} Demand Showing Filters'.format(region)
    save = '/Users/truggles/tmp_plots/{}_demand_show_filters.png'.format(region)
    scrolling_demand(width, dem_map[region], title, save)


o_file.close()